In [2]:
import numba
import numpy as np
from numba import cuda

@numba.njit
def func_cpu(a, b=0.):
    for i in range(a.shape[0]):
        a[i]+= b

@cuda.jit
def func_cuda(a, b=0.):
    for i in range(cuda.grid(1), a.shape[0], cuda.gridsize(1)):
        a[i]+= b


# these all work
a = np.random.randn(1000)
_ = func_cpu(a)
func_cpu(a, b=1.)
func_cpu(a, 2.)

a_g = cuda.to_device(a)
# 1. This works
func_cuda[4,256](a_g, 2.) 

# 2. Raises TypeError:
#    TypeError: Failed in cuda mode pipeline (step: fix up args)
#    Signature mismatch: 1 argument types given, but function takes 2 arguments
func_cuda[4,256](a_g) 

# 3. Raise TypeError:
#    __call__() got an unexpected keyword argument 'b'
func_cuda[4,256](a_g, b=1.)

SyntaxError: unmatched '}' (2584903666.py, line 30)

In [2]:
%timeit func_cuda[4,256](a_g, b=1.)

39.3 µs ± 382 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [3]:
%timeit func_cuda[4,256](a_g, b=1.)

38.6 µs ± 325 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [3]:
import pragma
import numba
fns = [sin, cos, tan]

@numba.jit
@pragma.deindex(fns, 'fns')
@pragma.unroll(num_fns=len(fns))
def call(i, x):
   for j in range(num_fns):
      if i == j:
         return fns[j](x)  # Still dynamic call, but decorators convert to static


ImportError: cannot import name 'operatorPrecedence' from 'pyparsing' (/home/tingkai/anaconda3/envs/neural/lib/python3.9/site-packages/pyparsing/__init__.py)